In [1]:
import pandas as pd

In [2]:
pd.options.display.max_rows=1000
pd.options.display.max_columns=1000

DATA_DIR = '/home/jupyter/kaggle/predict_future_sales/data/read_only'

In [3]:
def read_data(data_dir, train=True, 
              test=True, sample_submission=True, 
              items=True, item_categories=True, 
              shops=True):
    print('Reading Data...')
    train_df = None
    test_df = None
    sample_submission_df = None
    items_df = None
    item_categories_df = None
    shops_df = None
    
    if train:
        train_df = pd.read_csv(f'{data_dir}/sales_train.csv')
        print(f'Shape of train_df : {train_df.shape}')
    if test:
        test_df = pd.read_csv(f'{data_dir}/test.csv')
        print(f'Shape of test_df : {test_df.shape}')
    if sample_submission:
        sample_submission_df = pd.read_csv(f'{data_dir}/sample_submission.csv')
        print(f'Shape of sample_submission_df : {sample_submission_df.shape}')
    if items:
        items_df = pd.read_csv(f'{data_dir}/items.csv')
        print(f'Shape of items_df : {items_df.shape}')
    if item_categories:
        item_categories_df = pd.read_csv(f'{data_dir}/item_categories.csv')
        print(f'Shape of item_categories_df : {item_categories_df.shape}')
    if shops:
        shops_df = pd.read_csv(f'{data_dir}/shops.csv')
        print(f'Shape of shops_df : {shops_df.shape}')    
    
    return train_df, test_df, sample_submission_df, items_df, item_categories_df, shops_df


def get_boundary_date_string(date_sr, boundary, date_format='%Y-%m-%d'):
    """
    Returns the first or last date of the series date_sr based on the 
    value passed in boundary. 
    
    date_sr: Series consisting of date
    boundary: Allowed values are 'first' or 'last'
    date_format: Format to be used for converting date into String
    """
    return date_sr.describe().loc[boundary].strftime(format='%Y-%m-%d')

    
def get_first_date_string(date_sr, date_format='%Y-%m-%d'):
    """
    Returns the first date of the series date_sr
    
    date_sr: Series consisting of date
    date_format: Format to be used for converting date into String
    """
    return get_boundary_date_string(date_sr, boundary='first', date_format='%Y-%m-%d')


def get_last_date_string(date_sr, date_format='%Y-%m-%d'):
    """
    Returns the last date of the series date_sr

    date_sr: Series consisting of date
    date_format: Format to be used for converting date into String
    """
    return get_boundary_date_string(date_sr, boundary='last', date_format='%Y-%m-%d')

    
def find_missing_dates(date_sr, start_date, end_date):
    """
    Returns the dates which are missing in the series
    date_sr between the start_date and end_date
    
    date_sr: Series consisting of date
    start_date: Start date in String format
    end_date: End date in String format
    """
    return pd.date_range(
        start=start_date, end=end_date).difference(date_sr)


def merge_df(left_df, right_df, how, on):
    """
    Wrapper on top of Pandas merge. Prints the shape & missing 
    values before and after merge
    """
    print(f"Before merge missing values on left_df")
    print(left_df.isna().sum())
    print(f"Before merge missing values on right_df")
    print(right_df.isna().sum())
    print(f"Before merge shape of left_df: {left_df.shape}")
    print(f"Before merge shape of right_df: {right_df.shape}")
    merged_df = pd.merge(left_df, right_df, how=how, on=on)
    print(f"After merge missing values in merged_df")
    print(merged_df.isna().sum())
    print(f"After merge shape of merged_df {merged_df.shape}")
    return merged_df

def prepare_data(train_df, items_df, item_categories_df, shops_df):
    item_item_categories_df = merge_df(items_df, item_categories_df, how='outer', on='item_category_id')
    print("Merging train_df with items ....")
    train_df_item_merged_df = merge_df(train_df, item_item_categories_df, how='left', on='item_id')
    print("Merging train_df with shops ....")
    train_df_item_shop_merged_df = merge_df(train_df_item_merged_df, shops_df, how='left', on='shop_id')
    return train_df_item_shop_merged_df

In [4]:
train_df, test_df, sample_submission_df, items_df, item_categories_df, shops_df = read_data(DATA_DIR)
# Format the date format. This is preferred compared to auti infer
train_df.date = pd.to_datetime(train_df.date, format='%d.%m.%Y')
# Sort train data based on date
train_df.sort_values(by='date', inplace=True)

Reading Data...
Shape of train_df : (2935849, 6)
Shape of test_df : (214200, 3)
Shape of sample_submission_df : (214200, 2)
Shape of items_df : (22170, 3)
Shape of item_categories_df : (84, 2)
Shape of shops_df : (60, 2)


## Explore train_df

In [5]:
train_df.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
49800,2013-01-01,0,18,5823,2500.0,1.0
29784,2013-01-01,0,27,5573,849.0,1.0
35476,2013-01-01,0,7,1006,399.0,1.0
8330,2013-01-01,0,19,17707,899.0,1.0
57384,2013-01-01,0,14,19548,149.0,1.0


In [6]:
# Any nulls?
train_df.isna().sum()

date              0
date_block_num    0
shop_id           0
item_id           0
item_price        0
item_cnt_day      0
dtype: int64

In [7]:
# Is there any duplicate?
train_df.duplicated(subset=['date', 'shop_id', 'item_id'], keep=False).sum()

56

In [8]:
# Check if there are duplicates?
train_df[train_df.duplicated(subset=['date', 'shop_id', 'item_id'], keep=False)].head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
76961,2013-01-05,0,54,20130,149.0,1.0
76962,2013-01-05,0,54,20130,149.0,1.0
70718,2013-01-25,0,31,14050,248.0,1.0
70686,2013-01-25,0,31,14050,349.0,1.0
103463,2013-01-27,0,38,15702,149.0,-1.0


In [9]:
train_df.date.describe()

count                 2935849
unique                   1034
top       2013-12-28 00:00:00
freq                     9434
first     2013-01-01 00:00:00
last      2015-10-31 00:00:00
Name: date, dtype: object

In [10]:
train_df.groupby('date').shop_id.nunique().describe()

count    1034.000000
mean       45.675048
std         2.999325
min        17.000000
25%        44.000000
50%        45.000000
75%        48.000000
max        52.000000
Name: shop_id, dtype: float64

In [11]:
train_df.date.nunique()

1034

In [12]:
# Find if there is any date missing?
find_missing_dates(train_df.date, 
                   start_date=get_first_date_string(train_df.date), 
                   end_date=get_last_date_string(train_df.date))

DatetimeIndex([], dtype='datetime64[ns]', freq=None)

In [13]:
# How many shops for which training data is there?
train_df.shop_id.nunique()

60

In [14]:
# How many items for which training data is there?
train_df.item_id.nunique()

21807

## Explore Shops

In [33]:
shops_df.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [34]:
shops_df.tail()

,shop_name,shop_id
55,Цифровой склад 1С-Онлайн,55
56,"Чехов ТРЦ ""Карнавал""",56
57,"Якутск Орджоникидзе, 56",57
58,"Якутск ТЦ ""Центральный""",58
59,"Ярославль ТЦ ""Альтаир""",59


In [16]:
shops_df.isna().sum()

shop_name    0
shop_id      0
dtype: int64

In [17]:
shops_df.nunique()

shop_name    60
shop_id      60
dtype: int64

In [18]:
shops_df[shops_df.duplicated(['shop_name', 'shop_id'], keep=False)].shape

(0, 2)

## Explore Item Categories

In [19]:
item_categories_df.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [20]:
item_categories_df.nunique()

item_category_name    84
item_category_id      84
dtype: int64

In [21]:
item_categories_df.isna().sum()

item_category_name    0
item_category_id      0
dtype: int64

## Explore Items

In [22]:
items_df.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [23]:
items_df.isna().sum()

item_name           0
item_id             0
item_category_id    0
dtype: int64

In [24]:
items_df.item_id.nunique()

22170

In [25]:
items_df.item_category_id.nunique()

84

In [26]:
items_df.groupby('item_category_id')['item_id'].nunique().sum()

22170

## Explore Test Data

In [27]:
test_df.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [28]:
test_df.shop_id.nunique()

42

In [29]:
test_df.item_id.nunique()

5100

In [30]:
# For each shop, how many items that need to be predicted
test_df.groupby('shop_id').item_id.nunique()

shop_id
2     5100
3     5100
4     5100
5     5100
6     5100
7     5100
10    5100
12    5100
14    5100
15    5100
16    5100
18    5100
19    5100
21    5100
22    5100
24    5100
25    5100
26    5100
28    5100
31    5100
34    5100
35    5100
36    5100
37    5100
38    5100
39    5100
41    5100
42    5100
44    5100
45    5100
46    5100
47    5100
48    5100
49    5100
50    5100
52    5100
53    5100
55    5100
56    5100
57    5100
58    5100
59    5100
Name: item_id, dtype: int64

### Train Data : Let's do a small test to understand if the items are the same across all the stores or not.

I compare the items in Store 5 with all other stores. If there is any differnce it should be reflected here.

In [36]:
train_shop_list = list(train_df.shop_id.unique())

items_in_shop_5 = set(train_df[train_df.shop_id == 5]['item_id'].unique())
for shop in train_shop_list:
    diff = len(set(train_df[train_df.shop_id == shop]['item_id'].unique()) - items_in_shop_5)
    print(f'For shop {shop} number of items different from shop 5 are {diff}')

For shop 18 number of items different from shop 5 are 3398
For shop 27 number of items different from shop 5 are 4585
For shop 7 number of items different from shop 5 are 2403
For shop 19 number of items different from shop 5 are 3332
For shop 14 number of items different from shop 5 are 2231
For shop 8 number of items different from shop 5 are 738
For shop 28 number of items different from shop 5 are 5847
For shop 51 number of items different from shop 5 are 3111
For shop 54 number of items different from shop 5 are 7551
For shop 42 number of items different from shop 5 are 5402
For shop 46 number of items different from shop 5 are 3019
For shop 37 number of items different from shop 5 are 3226
For shop 15 number of items different from shop 5 are 2516
For shop 41 number of items different from shop 5 are 1984
For shop 13 number of items different from shop 5 are 1542
For shop 38 number of items different from shop 5 are 2106
For shop 2 number of items different from shop 5 are 1117
F

### Train Data : For a single shop check if the number of items are the same across month

I compare the items in Store 5 across all the months

In [38]:
train_df['month'] = train_df.date.dt.month

In [40]:
train_df_5 = train_df[train_df.shop_id == 5]

In [41]:
train_df_5.groupby('month')['item_id'].nunique()

month
1     [15063, 7207, 7059, 7980, 14826, 16215, 16205,...
2     [21565, 15104, 4906, 7978, 2319, 2750, 1409, 1...
3     [22020, 21705, 13878, 12745, 19811, 14752, 174...
4     [12691, 14574, 18749, 19415, 19195, 4366, 2827...
5     [6156, 7860, 8283, 7888, 6575, 6699, 6611, 182...
6     [5921, 5272, 6652, 6954, 11856, 11854, 11870, ...
7     [13937, 5587, 5821, 6466, 6457, 6111, 6490, 64...
8     [6774, 7259, 7272, 7072, 2929, 4372, 972, 1421...
9     [14883, 15472, 15463, 15462, 15458, 15455, 154...
10    [5616, 6086, 1740, 3719, 3331, 14861, 16523, 1...
11    [20949, 20237, 21918, 22072, 13946, 13614, 107...
12    [13629, 6675, 7207, 7206, 7203, 7096, 6498, 64...
Name: item_id, dtype: object

### Test Data : Let's do a small test to understand if the items are the same across all the stores or not.

I compare the items in Store 5 with all other stores. If there is any differnce it should be reflected here.

In [31]:
test_shop_list = list(test_df.shop_id.unique())

items_in_shop_5 = set(test_df[test_df.shop_id == 5]['item_id'].unique())
for shop in test_shop_list:
    diff = len(set(test_df[test_df.shop_id == shop]['item_id'].unique()) - items_in_shop_5)
    print(f'For shop {shop} number of items different from shop 5 are {diff}')
    diff = len(items_in_shop_5 - set(test_df[test_df.shop_id == shop]['item_id'].unique()))
    print(f'For shop {shop} number of items different from shop 5 are {diff}')

For shop 5 number of items different from shop 5 are 0
For shop 5 number of items different from shop 5 are 0
For shop 4 number of items different from shop 5 are 0
For shop 4 number of items different from shop 5 are 0
For shop 6 number of items different from shop 5 are 0
For shop 6 number of items different from shop 5 are 0
For shop 3 number of items different from shop 5 are 0
For shop 3 number of items different from shop 5 are 0
For shop 2 number of items different from shop 5 are 0
For shop 2 number of items different from shop 5 are 0
For shop 7 number of items different from shop 5 are 0
For shop 7 number of items different from shop 5 are 0
For shop 10 number of items different from shop 5 are 0
For shop 10 number of items different from shop 5 are 0
For shop 12 number of items different from shop 5 are 0
For shop 12 number of items different from shop 5 are 0
For shop 28 number of items different from shop 5 are 0
For shop 28 number of items different from shop 5 are 0
For 

## Explore Submission

In [30]:
sample_submission_df.head()

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5


In [58]:
# ### Lets filter the training data based on the test data. 
# - Training data has 21807 unique items sold across 60 shops
# - Test data has 42 shops and 5100 items for each shop

# train_df_org = train_df.copy(deep=True)

# test_shop_list = list(test_df.shop_id.unique())
# test_items_list = list(test_df.item_id.unique())

# print(f'Shape of the training data before filtering {train_df.shape}')

# train_df = train_df[train_df.shop_id.isin(test_shop_list) & train_df.item_id.isin(test_items_list)]

# train_df.groupby('shop_id')['item_id'].nunique()

### Check if for individual shop all the items are sold on all the days

In [74]:
train_df_shop_25 = train_df[train_df.shop_id == 25]

train_df_shop_25.groupby('date')['item_id'].nunique()

date
2013-01-02     80
2013-01-03     54
2013-01-04     50
2013-01-05     45
2013-01-06     49
             ... 
2015-10-27     93
2015-10-28    105
2015-10-29    141
2015-10-30    184
2015-10-31    198
Name: item_id, Length: 1030, dtype: int64

In [78]:
train_df_shop_19 = train_df[train_df.shop_id == 19]

train_df_shop_19.groupby('date')['item_id'].nunique()

date
2013-01-01    17
2013-01-02    44
2013-01-03    44
2013-01-04    30
2013-01-05    27
              ..
2015-10-27    46
2015-10-28    49
2015-10-29    38
2015-10-30    43
2015-10-31    86
Name: item_id, Length: 1031, dtype: int64

### Check if across shops all the items are sold on all the days

In [76]:
train_df.groupby('date')['item_id'].nunique()

date
2013-01-01     129
2013-01-02     387
2013-01-03     356
2013-01-04     318
2013-01-05     345
              ... 
2015-10-27     827
2015-10-28     797
2015-10-29     897
2015-10-30    1086
2015-10-31    1253
Name: item_id, Length: 1034, dtype: int64

In [77]:
train_df_org.groupby('date')['item_id'].nunique()

date
2013-01-01    1117
2013-01-02    2788
2013-01-03    2726
2013-01-04    2499
2013-01-05    2504
              ... 
2015-10-27     907
2015-10-28     842
2015-10-29     952
2015-10-30    1180
2015-10-31    1349
Name: item_id, Length: 1034, dtype: int64

### Observation
- Across shops all the items are not sold on all the days
- For individual shops as well, all the items are not sold on all the days